### Import the Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, classification_report
from sklearn import datasets
from sklearn.datasets import load_breast_cancer

### Loading Data and Model Preparation

In [2]:
# Load dataset
cancer = datasets.load_breast_cancer()

# Convert to DataFrame
cancer_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
target_df = pd.DataFrame(cancer.target, columns=["Target"])

# Display the first few rows of the dataframe
cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
target_df.head()

,Target
0,0
1,0
2,0
3,0
4,0


In [4]:
# Checking if there are any null values in the dataset
null_values = cancer_df.isnull().sum()
print(null_values)

mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimeter            0
worst area                 0
worst smoothness           0
worst compactness          0
worst concavity            0
worst concave points       0
worst symmetry             0
worst fractal dimension    0
dtype: int64


In [5]:
# Spliti into features and labels
X = cancer.data  
y = cancer.target

# Splitting dataset into the training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=109)

## k-fold cross-validation to estimate the model's performance without hyperparameter tuning

In [6]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Initialize the SVM model
model = SVC(random_state=109)

# Define a StratifiedKFold cross-validator for k-fold cross-validation
stratified_kfold_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=109)

# Perform k-fold cross-validation using the stratified_kfold_cv object
cv_scores = cross_val_score(model, X_train, y_train, cv=stratified_kfold_cv)

# Calculate the mean of the cross-validation scores
cv_scores_mean = np.mean(cv_scores)

print(f"Mean cross-validation score: {cv_scores_mean}")

Mean cross-validation score: 0.8968987341772152


## Hyperparameter Tuning using GridSearchCV with StratifiedKFold 

In [7]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for SVC
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'kernel': ['rbf', 'linear']
}

# Initialize the GridSearchCV object with the stratified_kfold_cv object for cross-validation
grid_search = GridSearchCV(SVC(random_state=109), param_grid, cv=stratified_kfold_cv, scoring='accuracy', refit=True)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters and score from GridSearchCV
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {best_score}")

Best parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Best cross-validation score: 0.9497151898734177


##  Evaluation on Test Dataset

In [8]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

y_pred = grid_search.best_estimator_.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate sensitivity (recall)
sensitivity = recall_score(y_test, y_pred)

# Calculate specificity from the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
TN, FP, FN, TP = conf_matrix.ravel()
specificity = TN / (TN + FP)

# Print the metrics
print('Accuracy:', accuracy)
print('Sensitivity:', sensitivity)
print('Specificity:', specificity)

Accuracy: 0.9649122807017544
Sensitivity: 0.9629629629629629
Specificity: 0.9682539682539683
